### Opening and Reading Pdf

In [9]:
import PyPDF2
import pprint
import re
from tabula import read_pdf
import pandas as pd
from pdfminer.high_level import extract_pages, extract_text, extract_text_to_fp

In [10]:
pdf_path = 'EN-FINAL Table 9.pdf'
pdf_txt = 'en-final-table9.txt'

In [11]:
#parsing the first two pages of the pdf and extracting their text
with open(pdf_path, 'rb') as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    num_pages = len(reader.pages)

    for page_number in range(2):
        page = reader.pages[page_number]
        text = page.extract_text()
        print(text)

TABLE 9
Afghanistan 10  11  10  15  40  37  –  –  –  –  90  74  75  74 Albania 12  14  9  0  10  99  –  –  –  36  30  75  78  71 Algeria 5 y 6 y 4 y 0  2  99  –  –  –  –  68  88  89  87 Andorra –   –  –  –  –  100 v –  –  –  –  –  –  –  – Angola 24 x 22 x 25 x –  –  36 x –  –  –  –  –  –  –  – Antigua and Barbuda –   –   –    –  –  –  –  –  –  –  –  –  –  – Argentina 7 y 8 y 5 y –  –  99 y –  –  –  –  –  –  –  – Armenia 4  5  3  0  7  100  –  –  –  20  9  70  72  67 Australia –   –   –    –  –  100 v –  –  –  –  –  –  –  – Austria –   –   –    –  –  100 v –  –  –  –  –  –  –  – Azerbaijan 7 y 8 y 5 y 1  12  94  –  –  –  58  49  75  79  71 Bahamas –   –   –    –  –  –  –  –  –  –  –  –  –  – Bahrain 5 x 6 x 3 x –  –  –  –  –  –  –  –  –  –  – Bangladesh 13    18    8    29  65  31  –  –  –  –  33 y –  –  – Barbados –   –   –    –  –  –  –  –  –  –  –  –  –  – Belarus 1    1    2    0  3  100 y –  –  –  4  4  65 y 67 y 62 y Belgium –   –   –    –  –  100 v –  –  –  –  –  –  –  – Belize 6

### PDF to text

In [ ]:
######### Extracting text #########
text = extract_text(pdf_path)
text = re.sub(r'\n\s*\n', '\n', text)

with open(pdf_txt, "a") as file:
    file.write(text)

### Extracting the data from text file

In [12]:
def turn_on_off(line, status, start, prev_line, end='\n'):
    """
        this function checks to see if a line starts/ends with a certain
        value. If the line starts/ends with that value, the status is 
        set to on/off (True/False) as long as the previous line isn't special.
    """
    if line.startswith(start):
        status = True
    elif status:
        if line == end and prev_line != 'and areas':
            status = False
    return status

In [13]:
def clean(line):
    """
        Cleans line breaks, spaces, and special characters from our line.
    """
    line = line.strip('\n').strip()
    line = line.replace('\xe2\x80\x93', "-")
    line = line.replace('\xe2\x80\x99', "'")
    return line

In [14]:
openfile = open(pdf_txt, 'r')

double_lined_countries = [
    'Bolivia (Plurinational',
    "Democratic People's",
    'Democratic Republic',
    "Lao People's Democratic",
    'Micronesia (Federated',
    'Saint Vincent and',
    'The former Yugoslav',
    'United Republic',
    'Venezuela (Bolivarian'
]

countries = []
totals = []
country_line = total_line = False
previous_line = ''

for line in openfile:
    
    if country_line:
        if previous_line.rstrip() in double_lined_countries:
            line = ' '.join([clean(previous_line), clean(line)])
            countries.append(line)
        elif line.rstrip() not in double_lined_countries:
            countries.append(clean(line))
    elif total_line:
        if len(line.replace('\n', '').strip()) > 0:
            totals.append(clean(line))
        
    country_line = turn_on_off(line, country_line, 'and areas', previous_line)
    total_line = turn_on_off(line, total_line, 'total', previous_line)
    
    previous_line = line
    
data = dict(zip(countries, totals))
pprint.pprint(data)

{'': '54  y',
 'Afghanistan': '10',
 'Albania': '12',
 'Algeria': '5  y',
 'Andorra': 'â€“',
 'Angola': '24  x',
 'Antigua and Barbuda': 'â€“',
 'Argentina': '7  y',
 'Armenia': '4',
 'Australia': 'â€“',
 'Austria': 'â€“',
 'Azerbaijan': '7  y',
 'Bahamas': 'â€“',
 'Bahrain': '5  x',
 'Bangladesh': '13',
 'Barbados': 'â€“',
 'Belarus': '1',
 'Belgium': 'â€“',
 'Belize': '6',
 'Benin': '46',
 'Bhutan': '3',
 'Bolivia (Plurinational State of)': '37',
 'Bosnia and Herzegovina': '99',
 'Botswana': '99',
 'Brazil': '100 v',
 'Brunei Darussalam': '36 x',
 'Bulgaria': 'â€“',
 'Burkina Faso': '99 y',
 'Burundi': '100',
 'Cabo Verde': '100 v',
 'Cambodia': '100 v',
 'Cameroon': '94',
 'Canada': 'â€“',
 'Central African Republic': 'â€“',
 'Chad': '31',
 'Chile': 'â€“',
 'China': '100 y',
 'Colombia': '100 v',
 'Comoros': '95',
 'Congo': '80',
 'Cook Islands': '74',
 'Costa Rica': '75',
 'Croatia': 'â€“',
 'Cuba': 'â€“',
 'Cyprus': 'â€“',
 'Czech Republic': 'â€“',
 'CÃ´te dâ€™Ivoire': '88',
 'Dem